In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error

def naive_forecast(train, test):
    return np.full(test.shape, train.iloc[-1])

def naive_seasonal(train, test, season_length=12):
    return np.tile(train[-season_length:], len(test) // season_length + 1)[:len(test)]

def cumulative_forecast(train, test):
    cumulative_mean = train.mean()
    return np.full(test.shape, cumulative_mean)

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate(y_true, y_pred):
    not_nan_indices = ~np.isnan(y_pred)
    y_true_filtered = y_true[not_nan_indices]
    y_pred_filtered = y_pred[not_nan_indices]
    
    rmse = np.sqrt(mean_squared_error(y_true_filtered, y_pred_filtered))
    mae = mean_absolute_error(y_true_filtered, y_pred_filtered)
    mape_val = mape(y_true_filtered, y_pred_filtered)
    
    return rmse, mae, mape_val


def plot_data(train, test, predictions, title):
    plt.figure(figsize=(12, 6))
    plt.plot(train.index, train, label='Train')
    plt.plot(test.index, test, label='Test')
    plt.plot(test.index, predictions, label='Predictions')
    plt.title(title)
    plt.legend()
    plt.show()

# Load the data
data = pd.read_csv("FULL_DATASET_.csv")

# Split the data into training and testing data
# 80% training data and 20% testing data was used for analysis
train = data.loc[data["Period"] < 41, "Services"]
test = data.loc[data["Period"] >= 41, "Services"]

# Initialize models
arima = ARIMA(train, order=(1, 1, 1)).fit()
sarima = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)).fit()
ses = SimpleExpSmoothing(train).fit()
ets_t = ExponentialSmoothing(train, trend='add').fit()
ets_t_s = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12).fit()
ets_s = ExponentialSmoothing(train, seasonal='add', seasonal_periods=12).fit()

# Print the optimized parameters for each model
print("Simple Exponential Smoothing Model:")
print(f"Alpha (smoothing level): {ses.params['smoothing_level']:.4f}\n")

print("Exponential Smoothing with Trend:")
print(f"Alpha (level): {ets_t.params['smoothing_level']:.4f}")
print(f"Gamma (trend): {ets_t.params['smoothing_trend']:.4f}\n")

print("Exponential Smoothing with Trend and Seasonality:")
print(f"Alpha (level): {ets_t_s.params['smoothing_level']:.4f}")
print(f"Gamma (trend): {ets_t_s.params['smoothing_trend']:.4f}")
print(f"Delta (seasonality): {ets_t_s.params['smoothing_seasonal']:.4f}\n")

print("Exponential Smoothing with Seasonality and Level:")
print(f"Alpha (level): {ets_s.params['smoothing_level']:.4f}")
print(f"Delta (seasonality): {ets_s.params['smoothing_seasonal']:.4f}\n")

# Print the parameters for the ARIMA model
print("ARIMA Model:")
print(f"p: {arima.model.order[0]}, d: {arima.model.order[1]}, q: {arima.model.order[2]}\n")

# Print the parameters for the SARIMA model
print("SARIMA Model:")
print(f"p: {sarima.model.order[0]}, d: {sarima.model.order[1]}, q: {sarima.model.order[2]}")
print(f"P: {sarima.model.seasonal_order[0]}, D: {sarima.model.seasonal_order[1]}, Q: {sarima.model.seasonal_order[2]}, s: {sarima.model.seasonal_order[3]}\n")

# Make predictions
arima_pred = arima.predict(start=len(train), end=len(train)+len(test)-1)
sarima_pred = sarima.predict(start=len(train), end=len(train)+len(test)-1)
ses_pred = ses.forecast(len(test))
naive_pred = naive_forecast(train, test)
naive_seasonal_pred = naive_seasonal(train, test)
cumulative_pred = cumulative_forecast(train, test)
ets_t_pred = ets_t.forecast(len(test))
ets_t_s_pred = ets_t_s.forecast(len(test))
ets_s_pred = ets_s.forecast(len(test))

# Evaluate and print errors
models = {
    'ARIMA': arima_pred,
    'SARIMA': sarima_pred,
    'Simple Exponential Smoothing': ses_pred,
    'Naive Forecast': naive_pred,
    'Naive Forecast with Seasonality': naive_seasonal_pred,
    'Cumulative Forecast': cumulative_pred,
    'Exponential Smoothing with Trend': ets_t_pred,
    'Exponential Smoothing with Trend and Seasonality': ets_t_s_pred,
    'Exponential Smoothing with Seasonality and Level': ets_s_pred
}

for model_name, predictions in models.items():
    rmse, mae, mape_val = evaluate(test, predictions)
    print(f"{model_name} Results:")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape_val:.2f}")

    plot_data(train, test, predictions, model_name)

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Model', 'RMSE', 'MAE', 'MAPE'])

# Iterate through each model, evaluate it, and store the results in the DataFrame
for model_name, predictions in models.items():
    rmse, mae, mape_val = evaluate(test, predictions)
    results_df = results_df.append({'Model': model_name,
                                     'RMSE': rmse,
                                     'MAE': mae,
                                     'MAPE': mape_val,}, ignore_index=True)

# Save the results DataFrame to an Excel file
results_df.to_excel('time_series_testing_results_FULL_DATASET_.xlsx', index=False)
